In [1]:
# Import essentials
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os

In [2]:
train_df = pd.read_excel('ub-politician-tweets-ii/train-20190402-08_55_19.xlsx') # Load the `train` file

In [3]:
train_df.head()

,Id,username,party,text,created_at,retweet_count,favorite_count
0,0,Jaumeasens,bcomu,Que el Suprem no deixi comparèixer davant el P...,2019-01-22 11:09:09,205,378
1,1,subirats9,bcomu,Un artículo a leer de @ainagallego sobre un te...,2018-12-12 10:20:00,2,2
2,2,subirats9,bcomu,El sentiment de desprotecció davant les incert...,2018-11-24 23:06:50,20,58
3,3,neusmunte,jxcat,I cap d’ells te traça amb els micròfons i les ...,2018-10-23 20:13:25,161,462
4,4,cuevillasjacs,jxcat,El @sergimiquel s’ha acomiadat avui com a Secr...,2019-03-17 18:45:50,58,238


In [4]:
train_df.party[15]


'bcncapital'

In [24]:
#!pip install stop_words

In [5]:
import re
from stop_words import get_stop_words

def special_char(text):
    """Retrieve the special characters
    """
    return re.sub(r'\W', ' ', text)

def filter_single(text):
    """remove all single characters
    """
    return re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

def filter_singleStart(text):
    """Remove single characters from the start
    """
    return re.sub(r'\^[a-zA-Z]\s+', ' ', text)

def filter_multiplespace(text):
    """Substituting multiple spaces with single space
    """
    return re.sub(r'\s+', ' ', text, flags=re.I)

def stop_words():
    """Retrieve the stop words for vectorization -Feel free to modify this function
    """
    return get_stop_words('es') + get_stop_words('ca') + get_stop_words('en')

def filter_mentions(text):
    """Utility function to remove the mentions of a tweet
    """
    return re.sub("@\S+", "", text)

def filter_hashtags(text):
    """Utility function to remove the hashtags of a tweet
    """
    return re.sub("#\S+", "", text)

def filter_symb_hashtag(text):
    """Utility function to remove the hashtags symbol of a tweet
    """
    return re.sub("#", "", text)

def filter_symb1(text):
    """Utility function to remove special characters of a tweet
    """
    return re.sub(r'[(:+*?¿!¡.,;`"")]', "", text)

#def translate_text(text):
#    """Utility function to translate the text of a tweet
#    """
#    return translator.translate(text, dest='es').text

def lower_text(text):
    """Utility function to lower text
    """
    return text.lower()

def eliminate_numbers(text):
    """Utility function to lower text
    """
    return re.sub(r'\w*\d\w*', '', text).strip()

def eliminate_emojiis(text):
    """Utility to eliminate emojiis from the text
    """
    em_pat=re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return  em_pat.sub(r'',text)

### Pre-Processing (1-3 fets)
1) eliminate symbols :,?¿!';-
2) eliminate single characters  
2b) numbers (eliminate numbers and words with numbers)  
3) all to lower  
4) accents? (amb unidecode, pero passa ç a c, ñ a n, aixo introdueix errors?)  
5) i is fessim una columna amb 0-1 indicant si hi ha emoji? (sigui com sigui, pero algu potser els fa servir bastant?)
6) segurament, si hi ha hashtag del partit l'ha posat el mateix partit (publicitat)

In [6]:
train_df['text_clean'] = [filter_symb1(filter_symb_hashtag(eliminate_emojiis(filter_single(eliminate_numbers(lower_text(row['text'])))))) for index, row in train_df.iterrows()]
train_df.head(5)

,Id,username,party,text,created_at,retweet_count,favorite_count,text_clean
0,0,Jaumeasens,bcomu,Que el Suprem no deixi comparèixer davant el P...,2019-01-22 11:09:09,205,378,que el suprem no deixi comparèixer davant el p...
1,1,subirats9,bcomu,Un artículo a leer de @ainagallego sobre un te...,2018-12-12 10:20:00,2,2,un artículo leer de @ainagallego sobre un tema...
2,2,subirats9,bcomu,El sentiment de desprotecció davant les incert...,2018-11-24 23:06:50,20,58,el sentiment de desprotecció davant les incert...
3,3,neusmunte,jxcat,I cap d’ells te traça amb els micròfons i les ...,2018-10-23 20:13:25,161,462,i cap d’ells te traça amb els micròfons les gr...
4,4,cuevillasjacs,jxcat,El @sergimiquel s’ha acomiadat avui com a Secr...,2019-03-17 18:45:50,58,238,el @sergimiquel s’ha acomiadat avui com secret...


In [7]:
print (train_df.text[12])
print (train_df.text_clean[12])

Siempre en nuestro corazón y en nuestra memoria 🌈🌹
„Sea cual sea su voto, quiero decirle que los homosexuales no somos orientaciones sexuales que vagamos por el espacio: somos sus hijos, sus hermanos, sus compañeros de grupo, sus colegas de partido...“ #Zerolo 
siempre en nuestro corazón en nuestra memoria 
„sea cual sea su voto quiero decirle que los homosexuales no somos orientaciones sexuales que vagamos por el espacio somos sus hijos sus hermanos sus compañeros de grupo sus colegas de partido“ zerolo


### Afegim una columna amb idioma
tot i que alguns resultats ens han mostrat pitjors scores

### coses per fer:
- lemmatization: sino ho podem traduir (el googletranslator dona problemes) no el podem aplicar (no n'hi ha en catala)  
- identificar hastags? ara per ara han quedat

# Models
Ho fem fent servir el pipeline (aixo ens permet aplicar una funcio per buscar parametres optims)

## Logistic Regression
### Resultats
1.  score = 0.72 eliminant simbols, fent minuscula
{'clf__C': 10, 'clf__dual': True, 'clf__fit_intercept': True, 'clf__penalty': 'l2', 'clf__solver': 'liblinear', 'clf__tol': 0.0001, 'vect__max_features': 6000, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__stop_words':
2.  score = 0.72. Eliminant simbol hashtag  
{'clf__C': 10, 'clf__dual': False, 'clf__fit_intercept': True, 'clf__penalty': 'l2', 'clf__solver': 'sag', 'clf__tol': 0.001, 'vect__max_features': 6000, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__stop_words'
3.  score = 0.72 Eliminant Emojis
{'clf__C': 10, 'clf__dual': True, 'clf__fit_intercept': True, 'clf__penalty': 'l2', 'clf__solver': 'liblinear', 'clf__tol': 0.0001, 'vect__max_features': 6000, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__stop_words':

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, LinearSVR, SVC


# clf=SVC(kernel='rbf', degree=6, C=10) -> 0.16
#clf=SVC(kernel='linear', degree=6, C=10) -> 0.7, variant degree a [1,6,12] i c[10, 100] es queda igual
#clf=SVC(kernel='poly', degree=6, C=10, gamma='auto') -> 0.16
#clf=SVC(kernel='sigmoid', degree=6, C=10, gamma='auto') -> 0.16
clf=SVC(kernel='linear', degree=6, C=10, coef0=4)

X_train, X_test, y_train, y_test = train_test_split(train_df['text_clean'], train_df['party'].values, test_size=0.20, random_state=345 )
vectorizer= TfidfVectorizer(min_df=1, ngram_range=(1,3), max_features=12000, stop_words=stop_words())

Xvec=vectorizer.fit_transform(X_train).toarray()
Xvec_test=vectorizer.transform(X_test).toarray()


clf.fit(Xvec,y_train)
clf.predict(Xvec_test)

print ("score = %3.2f" %(clf.score(Xvec_test,y_test)) )